In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
X = df.iloc[:, :-1].values

In [5]:
y = df.iloc[:,-1].values

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [27]:
import tensorflow
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras import Sequential

In [12]:
import keras_tuner as kt

In [29]:
def build_model(hp):
  model = Sequential()

  counter =0
  
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:

      model.add(Dense(
        hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
        activation= hp.Choice('activation', values=['relu', 'tanh', 'sigmoid']),
        input_dim =8
      ))
      model.add(Dropout(hp.Choice("dropout" + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    else:
      model.add(Dense(
        hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
        activation= hp.Choice('activation', values=['relu', 'tanh', 'sigmoid']),
      ))
      model.add(Dropout(hp.Choice("dropout" + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    counter= counter+1

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer= hp.Choice('optimizer', values=['rmsprop', 'adam', 'adadelta', 'sgd','nadam']),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model



In [30]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5, 
                        directory='mydir',
                        project_name="final1"
)

c:\Users\irbag\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.3571428656578064

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 12s


In [32]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 40,
 'activation': 'tanh',
 'dropout0': 0.1,
 'optimizer': 'nadam',
 'units1': 32,
 'dropout1': 0.7,
 'units2': 80,
 'dropout2': 0.3,
 'units3': 32,
 'dropout3': 0.2,
 'units4': 112,
 'dropout4': 0.5,
 'units5': 8,
 'dropout5': 0.1,
 'units6': 96,
 'dropout6': 0.3,
 'units7': 120,
 'dropout7': 0.3,
 'units8': 64,
 'dropout8': 0.6}

In [33]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\irbag\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 23 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [34]:
model.fit(X_train, y_train, epochs=200, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/200


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7008 - loss: 0.5536 - val_accuracy: 0.8052 - val_loss: 0.4571
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6912 - loss: 0.5698 - val_accuracy: 0.8052 - val_loss: 0.4619
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7204 - loss: 0.5265 - val_accuracy: 0.8052 - val_loss: 0.4567
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7288 - loss: 0.5337 - val_accuracy: 0.8117 - val_loss: 0.4610
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7512 - loss: 0.4895 - val_accuracy: 0.7922 - val_loss: 0.4640
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7410 - loss: 0.5254 - val_accuracy: 0.7987 - val_loss: 0.4601
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7246 - loss: 0.5298 - val_accuracy: 0.8052 - val_loss: 0.4591
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7712 - loss: 0.4927 - val_accuracy: 0.7857 - val_los